# Tutorial 1: End-to-End Pandas Workflow

This tutorial demonstrates a complete Pandas transformation workflow using the dual-engine TransformationRunner.

**Learning Objectives:**
- Configure and run transformations with `engine="pandas"`
- Use TransformationRunnerFromConfig for metadata-driven pipelines
- Work with local CSV files and Pandas DataFrames
- Understand configuration-driven transformations

**Prerequisites:**
- odibi_de_v2 installed (`pip install -e .`)
- pandas installed
- Sample CSV files (provided below)

## Step 1: Setup Sample Data

In [ ]:
import pandas as pd
import os
from pathlib import Path

# Create sample directory
data_dir = Path("./tutorial_data/pandas_workflow")
data_dir.mkdir(parents=True, exist_ok=True)

# Create sample sales data (bronze layer)
sales_data = pd.DataFrame({
    'transaction_id': [1, 2, 2, 3, 4, 5],  # Note: duplicate transaction 2
    'product': ['Widget A', 'Widget B', 'Widget B', 'Widget C', 'Widget A', 'Widget D'],
    'quantity': [10, 5, 5, 8, 15, 3],
    'price': [25.50, 40.00, 40.00, 15.75, 25.50, 100.00],
    'customer_id': [101, 102, 102, 103, 101, 104],
    'date': ['2024-01-01', '2024-01-01', '2024-01-01', '2024-01-02', '2024-01-02', '2024-01-03']
})

sales_data.to_csv(data_dir / "sales_bronze.csv", index=False)
print(f"✓ Created sample data: {data_dir / 'sales_bronze.csv'}")
print(f"\nBronze Data (Raw):")
print(sales_data)

## Step 2: Define Transformation Function

Create a simple transformation module that:
1. Removes duplicates
2. Calculates total revenue
3. Converts date to datetime

In [ ]:
%%writefile ./tutorial_data/pandas_workflow/silver_transformations.py
"""
Silver layer transformations for sales data.
"""
import pandas as pd
from typing import Dict, Any

def clean_sales_data(df: pd.DataFrame, constants: Dict[str, Any] = None) -> pd.DataFrame:
    """
    Clean and enrich sales data for silver layer.
    
    Args:
        df: Raw sales DataFrame from bronze layer
        constants: Optional configuration parameters
    
    Returns:
        Cleaned DataFrame with revenue calculation
    """
    # Remove duplicate transactions
    df_clean = df.drop_duplicates(subset=['transaction_id'])
    
    # Calculate revenue
    df_clean['revenue'] = df_clean['quantity'] * df_clean['price']
    
    # Convert date to datetime
    df_clean['date'] = pd.to_datetime(df_clean['date'])
    
    # Apply optional filters from constants
    if constants and 'min_revenue' in constants:
        df_clean = df_clean[df_clean['revenue'] >= constants['min_revenue']]
    
    return df_clean

def aggregate_by_product(df: pd.DataFrame, constants: Dict[str, Any] = None) -> pd.DataFrame:
    """
    Aggregate sales data by product for gold layer.
    
    Args:
        df: Cleaned sales DataFrame from silver layer
        constants: Optional configuration parameters
    
    Returns:
        Aggregated DataFrame with product-level metrics
    """
    aggregated = df.groupby('product').agg({
        'transaction_id': 'count',
        'quantity': 'sum',
        'revenue': 'sum'
    }).reset_index()
    
    aggregated.columns = ['product', 'total_transactions', 'total_quantity', 'total_revenue']
    
    # Calculate average revenue per transaction
    aggregated['avg_revenue_per_transaction'] = (
        aggregated['total_revenue'] / aggregated['total_transactions']
    )
    
    return aggregated.sort_values('total_revenue', ascending=False)

## Step 3: Configure Transformations

Create transformation configuration using TransformationConfig format.

In [ ]:
from odibi_de_v2.config import TransformationConfig

# Silver Layer: Clean sales data
silver_config = TransformationConfig(
    project="retail_analytics",
    plant="all_stores",
    asset="sales",
    module="tutorial_data.pandas_workflow.silver_transformations",
    function="clean_sales_data",
    input_table="./tutorial_data/pandas_workflow/sales_bronze.csv",
    target_table="./tutorial_data/pandas_workflow/sales_silver.csv",
    enabled=True,
    env="tutorial",
    layer="silver"
)

# Gold Layer: Aggregate by product
gold_config = TransformationConfig(
    project="retail_analytics",
    plant="all_stores",
    asset="product_summary",
    module="tutorial_data.pandas_workflow.silver_transformations",
    function="aggregate_by_product",
    input_table="./tutorial_data/pandas_workflow/sales_silver.csv",
    target_table="./tutorial_data/pandas_workflow/product_gold.csv",
    enabled=True,
    env="tutorial",
    layer="gold"
)

print("✓ Configurations created")
print(f"\nSilver config: {silver_config.module}.{silver_config.function}")
print(f"Gold config: {gold_config.module}.{gold_config.function}")

## Step 4: Run Silver Layer Transformation

Use TransformationRunnerFromConfig with `engine="pandas"`

In [ ]:
from odibi_de_v2.transformer import TransformationRunnerFromConfig
from odibi_de_v2.logger import MetadataManager, DynamicLogger
import sys

# Add tutorial_data to Python path
sys.path.insert(0, str(data_dir.parent))

# Initialize metadata manager (in-memory for tutorial)
metadata_mgr = MetadataManager(connection_string=None)  # Uses in-memory storage
logger = DynamicLogger("pandas_tutorial")

# Create runner for silver layer
silver_runner = TransformationRunnerFromConfig(
    config=silver_config,
    metadata_manager=metadata_mgr,
    logger=logger,
    engine="pandas"  # ← Explicitly use Pandas engine
)

# Run transformation
print("\n" + "="*60)
print("RUNNING SILVER LAYER TRANSFORMATION (Pandas Engine)")
print("="*60)

silver_runner.run()

# Verify output
silver_df = pd.read_csv(data_dir / "sales_silver.csv")
print("\n✓ Silver layer transformation complete!")
print(f"\nSilver Data (Cleaned):")
print(silver_df)
print(f"\nRows: Bronze={len(sales_data)} → Silver={len(silver_df)} (duplicates removed)")

## Step 5: Run Gold Layer Transformation

In [ ]:
# Create runner for gold layer
gold_runner = TransformationRunnerFromConfig(
    config=gold_config,
    metadata_manager=metadata_mgr,
    logger=logger,
    engine="pandas"
)

# Run transformation
print("\n" + "="*60)
print("RUNNING GOLD LAYER TRANSFORMATION (Pandas Engine)")
print("="*60)

gold_runner.run()

# Verify output
gold_df = pd.read_csv(data_dir / "product_gold.csv")
print("\n✓ Gold layer transformation complete!")
print(f"\nGold Data (Aggregated by Product):")
print(gold_df)
print(f"\nRows: Silver={len(silver_df)} → Gold={len(gold_df)} (aggregated by product)")

## Step 6: Using Constants for Configuration

In [ ]:
# Silver layer with revenue filter
silver_config_filtered = TransformationConfig(
    project="retail_analytics",
    plant="all_stores",
    asset="high_value_sales",
    module="tutorial_data.pandas_workflow.silver_transformations",
    function="clean_sales_data",
    input_table="./tutorial_data/pandas_workflow/sales_bronze.csv",
    target_table="./tutorial_data/pandas_workflow/sales_silver_filtered.csv",
    enabled=True,
    env="tutorial",
    layer="silver",
    constants={"min_revenue": 200.0}  # Only transactions with revenue >= 200
)

filtered_runner = TransformationRunnerFromConfig(
    config=silver_config_filtered,
    metadata_manager=metadata_mgr,
    logger=logger,
    engine="pandas"
)

print("\n" + "="*60)
print("RUNNING FILTERED TRANSFORMATION (min_revenue=200)")
print("="*60)

filtered_runner.run()

filtered_df = pd.read_csv(data_dir / "sales_silver_filtered.csv")
print(f"\n✓ Filtered transformation complete!")
print(f"\nHigh-Value Sales (Revenue >= 200):")
print(filtered_df[['transaction_id', 'product', 'quantity', 'revenue']])
print(f"\nRows: All={len(silver_df)} → Filtered={len(filtered_df)}")

## Summary

**What You Learned:**

1. ✓ **Pandas Engine Usage**: Set `engine="pandas"` in TransformationRunnerFromConfig
2. ✓ **CSV I/O**: Read from and write to local CSV files
3. ✓ **Medallion Architecture**: Bronze → Silver → Gold progression
4. ✓ **Configuration-Driven**: Used TransformationConfig for metadata
5. ✓ **Constants**: Passed runtime parameters via `constants` dictionary

**Key Takeaways:**
- The Pandas engine works with local files and Pandas DataFrames
- Transformation functions accept `(df, constants)` signature
- TransformationRunnerFromConfig handles I/O, logging, and metadata automatically
- Constants enable parameterized transformations without code changes

**Next Steps:**
- Tutorial 2: Function Registry and Decorators
- Tutorial 3: Hooks and Observability
- Tutorial 4: Complete Project Template

In [ ]:
# Cleanup (optional)
import shutil
# shutil.rmtree(data_dir)  # Uncomment to remove tutorial data